# OpenBnB MCP 서버 통합을 활용한 Semantic Kernel

이 노트북은 MCPStdioPlugin을 사용하여 실제 OpenBnB MCP 서버와 Semantic Kernel을 연동하여 실제 Airbnb 숙소를 검색하는 방법을 보여줍니다. LLM 액세스를 위해 Azure AI Foundry를 사용합니다. 환경 변수를 설정하려면 [설정 레슨](/00-course-setup/README.md)을 따라 진행할 수 있습니다.


## 필요한 패키지 가져오기


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP 플러그인 연결 생성하기

MCPStdioPlugin을 사용하여 [OpenBnB MCP 서버](https://github.com/openbnb-org/mcp-server-airbnb)에 연결합니다. 이 서버는 @openbnb/mcp-server-airbnb 패키지를 통해 Airbnb 검색 기능을 제공합니다.


## 클라이언트 생성

이 샘플에서는 Azure AI Foundry를 사용하여 LLM에 접근합니다. 환경 변수가 올바르게 설정되어 있는지 확인하세요.


## 환경 구성

Azure OpenAI 설정을 구성하세요. 다음 환경 변수가 설정되어 있는지 확인하세요:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP 통합 이해하기

이 노트북은 실제 Airbnb 검색 기능을 제공하는 **실제 OpenBnB MCP 서버**에 연결됩니다.

### 작동 방식:

1. **MCPStdioPlugin**: MCP 서버와 표준 입출력을 통해 통신합니다.
2. **실제 NPM 패키지**: `@openbnb/mcp-server-airbnb`를 npx를 통해 다운로드하고 실행합니다.
3. **실시간 데이터**: Airbnb API에서 실제 숙소 데이터를 반환합니다.
4. **기능 탐색**: 에이전트가 MCP 서버에서 사용할 수 있는 기능을 자동으로 탐색합니다.

### 사용 가능한 기능:

OpenBnB MCP 서버는 일반적으로 다음 기능을 제공합니다:
- **search_listings** - 위치와 조건에 따라 Airbnb 숙소를 검색
- **get_listing_details** - 특정 숙소에 대한 상세 정보 가져오기
- **check_availability** - 특정 날짜의 예약 가능 여부 확인
- **get_reviews** - 숙소에 대한 리뷰 가져오기
- **get_host_info** - 숙소 호스트에 대한 정보 가져오기

### 사전 준비 사항:

- 시스템에 **Node.js**가 설치되어 있어야 합니다.
- MCP 서버 패키지를 다운로드하기 위한 **인터넷 연결**이 필요합니다.
- **NPX**가 사용 가능해야 합니다 (Node.js에 기본 포함).

### 연결 테스트:

MCP 서버를 수동으로 테스트하려면 다음 명령을 실행할 수 있습니다:
```bash
npx -y @openbnb/mcp-server-airbnb
```

이 명령은 OpenBnB MCP 서버를 다운로드하고 시작하며, 이후 Semantic Kernel이 실제 Airbnb 데이터를 가져오기 위해 서버에 연결합니다.


## OpenBnB MCP 서버와 함께 에이전트 실행하기

이제 OpenBnB MCP 서버에 연결된 AI 에이전트를 실행하여 스톡홀름에서 성인 2명과 어린이 1명을 위한 실제 Airbnb 숙소를 검색합니다. 검색 기준을 변경하려면 `user_inputs` 목록을 자유롭게 수정하세요.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# 요약
축하합니다! Model Context Protocol (MCP)을 사용하여 실제 숙소 검색과 통합된 AI 에이전트를 성공적으로 구축하셨습니다:

## 사용된 기술:
- Semantic Kernel - Azure OpenAI를 활용한 지능형 에이전트 구축
- Azure AI Foundry - LLM 기능 및 채팅 완성 지원
- MCP (Model Context Protocol) - 표준화된 도구 통합
- OpenBnB MCP 서버 - 실제 Airbnb 검색 기능 제공
- Node.js/NPX - 외부 MCP 서버 실행

## 배운 내용:
- MCP 통합: Semantic Kernel 에이전트를 외부 MCP 서버와 연결
- 실시간 데이터 접근: 라이브 API를 통해 실제 Airbnb 숙소 검색
- 프로토콜 통신: 에이전트와 MCP 서버 간 표준 입출력 통신 활용
- 기능 탐색: MCP 서버에서 사용 가능한 기능 자동 탐색
- 스트리밍 응답: 실시간으로 함수 호출 캡처 및 로깅
- HTML 렌더링: 스타일이 적용된 테이블과 인터랙티브 디스플레이로 에이전트 응답 포맷팅

## 다음 단계:
- 추가 MCP 서버 통합 (날씨, 항공편, 레스토랑 등)
- MCP와 A2A 프로토콜을 결합한 멀티 에이전트 시스템 구축
- 사용자 데이터 소스를 위한 맞춤형 MCP 서버 생성
- 세션 간 지속적인 대화 메모리 구현
- MCP 서버 오케스트레이션과 함께 에이전트를 Azure Functions에 배포
- 사용자 인증 및 예약 기능 추가



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문 번역가에 의한 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.  
